In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [3]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [4]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1

#     max_distance = np.nanmax(distance)
#     for i in range(cities):#
#         distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 
        
    return distance 

In [5]:
# function to calculate priori attractiveness of a decision 
# desirablity /visibility = inverse of the distance between two nodes (local information)
def visibility (distance):
    vis = np.zeros(distance.shape, dtype = np.float64)
    for i in range(distance.shape[0]):
        for j in range(distance.shape[1]):
            if distance[i,j] != 0:
                vis[i,j] = 1.0/distance[i,j]
                
    return vis

In [6]:
#define class Agent (ant) and its function
class Agent:
    import numpy as np
    import pandas as pd
    import math
    
    def __init__(self, k, cities, distance): #initialization
        self.index = k
        
        #randomly distribute the ant
        c = np.random.randint(low = 1, high = cities)
        self.currCity = c  
        
        self.visited = [0, c] #start at depot
        self.cost = distance[0,c]
        self.unvisited = [x for x in range(1,cities) if x!=c] 
        self.transition_probs = []
        self.tour = [(0,c)]
    
    def reset_agent(self): #reset whenever starting new iteration => ant return to depot
        self.currCity = 0 
        self.visited = [0]
        self.cost = 0
        self.unvisited = [x for x in range(1,cities)]
        self.transition_probs = []
        self.tour = []
        
    # get current transition prob (of unvisited city)
    def get_trans_prob(self, tau_list, vis_list, j): #j in unvisited
        tau_part = math.pow(tau_list[self.currCity,j], ALPHA)
        vis_part = math.pow(vis_list[self.currCity,j], BETA)
        
        return tau_part*vis_part
    

    def move(self, distance, tau_list, vis_list):
        
        while (self.unvisited): #loop while the unvisited list is not empty
            for j in self.unvisited:
                self.transition_probs.append(self.get_trans_prob(tau_list, vis_list, j))
                
            s = np.sum(self.transition_probs)
            for j in range(len(self.transition_probs)): 
                self.transition_probs[j]= self.transition_probs[j] / s #normalize
            
            nextCity = np.random.choice(self.unvisited,p=self.transition_probs) #random select next city based on trans_prob
            
            self.tour.append((self.currCity, nextCity)) #update tour
            self.visited.append(nextCity) #update into visited
            self.unvisited.remove(nextCity) #remove from unvisited
            self.cost += distance[self.currCity, nextCity] #update length of the tour
            
            self.currCity=nextCity #update current position
            self.transition_probs=[] #reset transition prob
        
        
        #return to depot
        self.tour.append((self.currCity, 0)) #update tour
        self.cost += distance[self.currCity, 0] #update length of the tour
        

In [7]:
# 120 cities
cities = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance=di.values

# for i in range(cities):
#     distance[i,i] = 10000

In [8]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [9]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

In [10]:
#Parameters

In [11]:
iterations = 100 #number of iterations

# intensities 
intensity = 0.0001

# # of ants: default value = # of cities
# too many ants will reinforce not optimal solutions (visit un-optimal tour too often)
# too few ants would not produce cooperative effect due to pheromone decay
agents = cities #number of agents

# Relative weight of feedback: how greedy the algorithm is => EXPLOITATION EFFECT 
# alpha = 0 => extremly greedy (alpha >= 0) => control the influence of the amount of pheromone deposited 
ALPHA = 2

# Relative weight of problem information: how fast the ants is converge to a steady solution => EXPLORATION EFFECT
# beta >= 1 => control desirability of state transition (i.e. 1/d)
BETA = 3

# Trail increment: how much pheromone is deposited in between steps in the process
# ontrol the level of exploration undertaken by the ants
Q = 1

# Pheromone evaporation rate (Forgetting rate)
# small = slow evaporation => more amplification of the initial random fluctuation
# high = fast evaporation => evaporate quickly (less influence of memory on decision) => constantly doing random searches
RHO = 0.1

In [12]:
ds = visibility(distance)
taus = intensity * np.ones(distance.shape)

In [13]:
#main algorithm

best_cost = np.max(distance)*(cities+1) #upperbound
best_tour = []

#run iterations
colony = []
for k in range(agents):
    ant = Agent(k, cities, distance)
    colony.append(ant)

for it in range(iterations):
    
    delta_tau = np.zeros((agents, cities, cities))
    
    for k in range(agents):
        colony[k].move(distance, taus, ds)
        tour_k = colony[k].tour
        L_k = colony[k].cost    
        colony[k].reset_agent()
        
        #print(L_k,'   ', best_cost)
        #print(tour_k)
        
        if L_k < best_cost:
            best_cost = L_k
            best_tour = tour_k
            print(best_cost)
        
        for i in range(cities):
            for j in range(cities):
                if (i,j) in colony[k].tour:
                    delta_tau[k,i,j] = Q/L_k
        
    for i in range(cities):
        for j in range(cities):
            taus[i,j] = RHO*taus[i,j] + sum( [delta_tau[k,i,j] for k in range(agents)] )
            
print(best_tour)
print(best_cost)

13961
13625
13191
12448
11360
11331
[(0, 31), (31, 29), (29, 28), (28, 119), (119, 13), (13, 43), (43, 45), (45, 49), (49, 97), (97, 16), (16, 117), (117, 48), (48, 40), (40, 55), (55, 6), (6, 37), (37, 71), (71, 39), (39, 7), (7, 69), (69, 33), (33, 3), (3, 25), (25, 70), (70, 115), (115, 89), (89, 35), (35, 103), (103, 111), (111, 101), (101, 47), (47, 46), (46, 88), (88, 54), (54, 5), (5, 83), (83, 34), (34, 9), (9, 98), (98, 61), (61, 36), (36, 66), (66, 56), (56, 82), (82, 72), (72, 62), (62, 4), (4, 79), (79, 114), (114, 1), (1, 8), (8, 118), (118, 102), (102, 22), (22, 50), (50, 10), (10, 20), (20, 108), (108, 92), (92, 63), (63, 76), (76, 52), (52, 26), (26, 100), (100, 113), (113, 105), (105, 109), (109, 38), (38, 94), (94, 96), (96, 11), (11, 87), (87, 81), (81, 2), (2, 95), (95, 53), (53, 15), (15, 60), (60, 75), (75, 73), (73, 104), (104, 27), (27, 44), (44, 77), (77, 93), (93, 85), (85, 21), (21, 84), (84, 106), (106, 19), (19, 112), (112, 68), (68, 64), (64, 42), (42, 107

In [14]:
distance1.shape

(152, 152)